## 1.	Обучите простую рекуррентную нейронную сеть (без GRU/LSTM, без внимания) решать задачу дешифровки шифра Цезаря.

<br>1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на N каждой буквы).
Например, если N=2, то буква A переходит в букву C.
Можно поиграться с языком на выбор (немецкий, русский и т. д.).
<br>2. Создать архитектуру рекуррентной нейронной сети.
<br>3. Обучить её (вход — зашифрованная фраза, выход — дешифрованная фраза).
<br>4. Проверить качество модели.

In [7]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
# Алфавит
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
alphabet_size = len(alphabet)
char2idx = {c: i for i, c in enumerate(alphabet)}
idx2char = {i: c for i, c in enumerate(alphabet)}

In [3]:
def caesar_encrypt(text, shift):
    return ''.join(alphabet[(char2idx[c] + shift) % alphabet_size] for c in text)

In [4]:
def caesar_decrypt(text, shift):
    return ''.join(alphabet[(char2idx[c] - shift) % alphabet_size] for c in text)

In [12]:
def text_to_tensor(text):
    return torch.tensor([char2idx[c] for c in text], dtype=torch.long)

In [5]:
def tensor_to_text(tensor):
    return ''.join([idx2char[int(i)] for i in tensor])

In [6]:
# Генерация выборки
class CaesarDataset(Dataset):
    def __init__(self, phrases, shift):
        self.data = []
        for phrase in phrases:
            phrase = phrase.lower().replace(" ", "")
            encrypted = caesar_encrypt(phrase, shift)
            self.data.append((text_to_tensor(encrypted), text_to_tensor(phrase)))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return x, y

In [14]:
class CaesarRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim=16, hidden_dim=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        logits = self.fc(output)
        return logits

In [8]:
def train_model(model, dataloader, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out.view(-1, ALPHABET_SIZE), y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [9]:
def evaluate(model, dataset, shift):
    model.eval()
    for i in range(5):
        x, y = dataset[i]
        x = x.unsqueeze(0)  # добавим batch dim
        with torch.no_grad():
            output = model(x)
            prediction = output.argmax(-1).squeeze()
        print(f"Зашифровано:  {tensor_to_text(x.squeeze())}")
        print(f"Расшифровка:  {tensor_to_text(prediction)}")
        print(f"Ожидалось:    {tensor_to_text(y)}\n")

In [10]:
phrases = ["привет", "шифрование"]
shift = 3

In [13]:
dataset = CaesarDataset(phrases * 100, shift=shift)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [15]:
model = CaesarRNN(vocab_size=alphabet_size)
train_model(model, dataloader, epochs=15)

RuntimeError: stack expects each tensor to be equal size, but got [10] at entry 0 and [6] at entry 1